---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [ ]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

In [57]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [58]:
def answer_one():    
    return len(spam_data[spam_data['target'] == 1])/len(spam_data) * 100


In [59]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    cv = CountVectorizer()
    cv_fit = cv.fit_transform(X_train)
    tokens_and_counts = cv.get_feature_names()
    return max(tokens_and_counts, key=len)

In [61]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [62]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    cv = CountVectorizer()
    X_fit = cv.fit_transform(X_train)
    X_test_fit = cv.transform(X_test)
    mnb = MultinomialNB(alpha=0.1)
    mnb.fit(X_fit, y_train)
    predict_x = mnb.predict(X_test_fit)
    return roc_auc_score(y_test, predict_x)

In [41]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [191]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)

    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

    smallest_tf = pd.Series(sorted_tfidf_index[:20],index=feature_names[sorted_tfidf_index[:20]]).sort_values()
    largest_tf = pd.Series(sorted_tfidf_index[:-21:-1],index=feature_names[sorted_tfidf_index[:-21:-1]]).sort_values(ascending=False)
    print(list(smallest_tf.index))
    return (smallest_tf, largest_tf)


In [192]:
answer_four()

['aaniye', 'athletic', 'chef', 'companion', 'courageous', 'dependable', 'determined', 'diwali', 'exterminator', 'healer', 'listener', 'mornings', 'organizer', 'pest', 'psychiatrist', 'psychologist', 'pudunga', 'stylist', 'sympathetic', 'venaam']


(aaniye           689
 athletic        1046
 chef            1676
 companion       1833
 courageous      1948
 dependable      2150
 determined      2172
 diwali          2259
 exterminator    2586
 healer          3196
 listener        3919
 mornings        4332
 organizer       4715
 pest            4887
 psychiatrist    5176
 psychologist    5178
 pudunga         5184
 stylist         6197
 sympathetic     6305
 venaam          6877
 dtype: int64, yup          7328
 where        7089
 too          6598
 tick         6517
 thanx        6439
 thank        6434
 okie         4651
 ok           4646
 nite         4523
 lei          3848
 home         3293
 havent       3180
 er           2496
 done         2298
 blank        1300
 beerage      1214
 anytime       933
 anything      931
 645           533
 146tf150p     274
 dtype: int64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [92]:
def answer_five():
    cv = TfidfVectorizer(min_df=3)
    X_fit = cv.fit_transform(X_train)
    X_test_fit = cv.transform(X_test)
    mnb = MultinomialNB(alpha=0.1)
    mnb.fit(X_fit, y_train)
    predict_x = mnb.predict(X_test_fit)
    return roc_auc_score(y_test, predict_x)

In [93]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [156]:
def answer_six():
    return spam_data.loc[spam_data['target'] == 0, 'text'].str.len().mean(), spam_data.loc[spam_data['target'] == 1, 'text'].str.len().mean()

In [157]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [211]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [150]:
from sklearn.svm import SVC

def answer_seven():
    cv = TfidfVectorizer(min_df=5)
    X_fit = cv.fit_transform(X_train)
    X_test_fit = cv.transform(X_test)
    svm = SVC(C=10000)
    svm.fit(X_fit, y_train)
    predict_x = svm.predict(X_test_fit)
    return roc_auc_score(y_test, predict_x)

In [151]:
answer_seven()

0.94971605860482489

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [154]:
def answer_eight():    
    return spam_data.loc[spam_data['target'] == 0, 'text'].str.count('\d').mean(), spam_data.loc[spam_data['target'] == 1, 'text'].str.count('\d').mean()

In [155]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [160]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    cv = TfidfVectorizer(min_df=5,ngram_range=(1,3))
    X_fit = cv.fit_transform(X_train)
    X_test_fit = cv.transform(X_test)
    lvm = LogisticRegression(C=100)
    lvm.fit(X_fit, y_train)
    predict_x = lvm.predict(X_test_fit)
    return roc_auc_score(y_test, predict_x)

In [161]:
answer_nine()

0.95601667147683478

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [167]:
def answer_ten():
    a = spam_data.loc[spam_data['target'] == 0, 'text'].str.count('~\w|\W').mean()
    b = spam_data.loc[spam_data['target'] == 1, 'text'].str.count('~\w|\W').mean()
    return a,b

In [168]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [258]:
non_word_char_count = X_train.str.count('~\w|\W')
# print(non_word_char_count)
digit_count = X_train.str.count('\d')
# print(digit_count)
length_of_doc = X_train.str.len()
# print(length_of_doc)



In [321]:
def answer_eleven():
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    X_train_vect = add_feature(X_train_vect, 
                              [
                                  X_train.str.count('~\w|\W'),
                                  X_train.str.count('\d'),
                                  X_train.str.len()
                              ]
                              )
    X_test_vect = add_feature(X_test_vect,
                             [
                                  X_test.str.count('~\w|\W'),
                                  X_test.str.count('\d'),
                                  X_test.str.len()
                              ]
                             )
    model = LogisticRegression(C=100)
    model.fit(X_train_vect, y_train)

    predictions = model.predict(X_test_vect)
    score = roc_auc_score(y_test, predictions)
    
    feature_names = vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count']
    feature_names = np.array(feature_names)
    
    sorted_coef_index = model.coef_[0].argsort()
    return score,list(feature_names[sorted_coef_index[:10]]),list(feature_names[sorted_coef_index[:-11:-1]])

In [322]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])